# Operaciones de cadenas vectorizadas

Una fortaleza de Python es su relativa facilidad para manejar y manipular datos de cadenas. Pandas se basa en esto y proporciona un conjunto integral de operaciones de cadenas vectorizadas que se convierten en una pieza esencial del tipo de manipulación requerida cuando se trabaja con (léase: limpieza) datos del mundo real. En esta sección, veremos algunas de las operaciones de cadena de Pandas y luego veremos cómo usarlas para limpiar parcialmente un conjunto de datos muy desordenado de recetas recopiladas de Internet.

## Introducción a las operaciones con cadenas de Pandas 

Vimos en secciones anteriores cómo herramientas como NumPy y Pandas generalizan las operaciones aritméticas para que podamos realizar fácil y rápidamente la misma operación en muchos elementos de la matriz. Por ejemplo:

In [ ]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Esta vectorización de operaciones simplifica la sintaxis de operar en matrices de datos: ya no tenemos que preocuparnos por el tamaño o la forma de la matriz, sino por la operación que queremos que se realice. Para matrices de cadenas, NumPy no proporciona un acceso tan simple y, por lo tanto, está atascado usando una sintaxis de bucle más detallada:

In [ ]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Esto quizás sea suficiente para trabajar con algunos datos, pero se romperá si faltan valores.
Por ejemplo:

In [ ]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas incluye funciones para abordar tanto esta necesidad de operaciones de cadenas vectorizadas como para manejar correctamente los datos faltantes a través del atributo ``str`` de los objetos Pandas Series e Index que contienen cadenas. 

Entonces, por ejemplo, supongamos que creamos una Serie Pandas con estos datos:

In [ ]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

Ahora podemos llamar a un único método que pondrá en mayúsculas todas las entradas, mientras omite los valores faltantes:

In [ ]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

Usando la finalización de tabulación en este atributo ``str`` enumerará todos los métodos de cadena vectorizados disponibles para Pandas.

## Tablas de Métodos de Cadena Pandas

Si tiene una buena comprensión de la manipulación de cadenas en Python, la mayor parte de la sintaxis de cadenas de Pandas es lo suficientemente intuitiva como para que probablemente sea suficiente enumerar una tabla de métodos disponibles; Comenzaremos con eso aquí, antes de profundizar en algunas de las sutilezas. Los ejemplos de esta sección utilizan la siguiente serie de nombres:

In [ ]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Métodos similares a los métodos de cadena de Python.

Casi todos los métodos de cadena integrados de Python se reflejan en un método de cadena vectorizado de Pandas. Aquí hay una lista de pandas métodos ``str`` que reflejan los métodos de cadena de Python: 

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()``    | ``lower()``      | ``translate()``  | ``islower()``    | 
|``ljust()``  | ``upper()``      | ``startswith()`` | ``isupper()``    | 
|``rjust()``  | ``find()``       | ``endswith()``   | ``isnumeric()``  | 
|``center()`` | ``rfind()``      | ``isalnum()``    | ``isdecimal()``  | 
|``zfill()``  | ``index()``      | ``isalpha()``    | ``split()``      | 
|``strip()``  | ``rindex()``     | ``isdigit()``    | ``rsplit()``     | 
|``rstrip()`` | ``capitalize()`` | ``isspace()``    | ``partition()``  | 
|``lstrip()`` |  ``swapcase()``  |  ``istitle()``   | ``rpartition()`` |

Tenga en cuenta que estos tienen varios valores de retorno. Algo como ``lower()``, devuelve una serie de cadenas: 

In [ ]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Pero algunos otros devuelven números:

In [ ]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

O valores booleanos:

In [ ]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Aún otros devuelven listas u otros valores compuestos para cada elemento:

In [ ]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

Veremos más manipulaciones de este tipo de objeto de serie de listas a medida que continuamos nuestra discusión.

### Métodos usando expresiones regulares

Además, existen varios métodos que aceptan expresiones regulares para examinar el contenido de cada elemento de cadena y siguen algunas de las convenciones de la API de Python módulo ``re``: 

Llamada re.match()en cada elemento, devolviendo grupos coincidentes como cadenas. 

| Method | Description |
|--------|-------------|
| ``match()`` | Llamada ``re.match()`` en cada elemento, devolviendo un valor booleano. |
| ``extract()`` | Llamada ``re.match()`` en cada elemento, devolviendo grupos coincidentes como cadenas.|
| ``findall()`` | Llamada ``re.findall()`` en cada elemento  |
| ``replace()`` | Reemplace las ocurrencias de patrón con alguna otra cadena |
| ``contains()`` | Llamada ``re.search()`` en cada elemento, devolviendo un valor booleano |
| ``count()`` | Contar ocurrencias de patrón |
| ``split()``   |  Equivalente a ``str.split()``, pero acepta expresiones regulares  |
| ``rsplit()`` |  Equivalente a ``str.rsplit()``, pero acepta expresiones regulares  |


Con estos, puede hacer una amplia gama de operaciones interesantes. Por ejemplo, podemos extraer el primer nombre de cada uno solicitando un grupo contiguo de caracteres al comienzo de cada elemento:

In [ ]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

O podemos hacer algo más complicado, como encontrar todos los nombres que comienzan y terminan con una consonante, haciendo uso del principio de cadena (``^``) y fin de cadena (``$``) caracteres de expresiones regulares:

In [ ]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

La capacidad de aplicar de forma concisa expresiones regulares en ``Series`` o ``Dataframe`` entradas abre muchas posibilidades para el análisis y limpieza de datos.

### Métodos misceláneos ¶ 
Finalmente, hay algunos métodos misceláneos que permiten otras operaciones convenientes: 

| Method | Description |
|--------|-------------|
| ``get()`` | Indexar cada elemento  |
| ``slice()`` | Cortar cada elemento |
| ``slice_replace()`` | Reemplace el segmento en cada elemento con el valor pasado|
| ``cat()``      | Concatenar cadenas|
| ``repeat()`` | Repetir valores |
| ``normalize()`` | Devuelve la forma Unicode de cadena |
| ``pad()`` | Agregue espacios en blanco a la izquierda, a la derecha o a ambos lados de las cadenas|
| ``wrap()`` | Dividir cadenas largas en líneas con una longitud inferior a un ancho determinado|
| ``join()`` | Unir cadenas en cada elemento de la Serie con separador pasado|
| ``get_dummies()`` | extraer variables ficticias como un dataframe  |

#### Acceso y corte de elementos vectorizados 

Las operaciones ``get()`` y ``slice()`` en particular, permiten el acceso a elementos vectorizados desde cada matriz. Por ejemplo, podemos obtener una porción de los primeros tres caracteres de cada matriz usando ``str.slice(0, 3)``. Tenga en cuenta que este comportamiento también está disponible a través de la sintaxis de indexación normal de Python, por ejemplo, ``df.str.slice(0, 3)`` es equivalente a ``df.str[0:3]``: 


In [ ]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

Indexación vía ``df.str.get(i)`` y ``df.str[i]`` es igualmente similar. 

Los métodos ``get()`` y ``slice()``también le permiten acceder a los elementos de las matrices devueltas por ``split()``. Por ejemplo, para extraer el apellido de cada entrada, podemos combinar ``split()`` y ``get()``: 

In [ ]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### Variables indicadoras 

Otro método que requiere un poco de explicación adicional es el método ``get_dummies()``. Esto es útil cuando sus datos tienen una columna que contiene algún tipo de indicador codificado. Por ejemplo, podríamos tener un conjunto de datos que contenga información en forma de códigos, como A="nacido en Estados Unidos", B="nacido en el Reino Unido", C="le gusta el queso", D="le gusta el spam" : 

In [ ]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,info,name
0,B|C|D,Graham Chapman
1,B|D,John Cleese
2,A|C,Terry Gilliam
3,B|D,Eric Idle
4,B|C,Terry Jones
5,B|C|D,Michael Palin


La rutina ``get_dummies()`` le permite dividir rápidamente estas variables indicadoras en un ``DataFrame``:

In [ ]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Con estas operaciones como bloques de construcción, puede construir una variedad infinita de procedimientos de procesamiento de cadenas al limpiar sus datos. 

No profundizaremos más en estos métodos aquí, pero lo animo a leer ["Working with Text Data"](http://pandas.pydata.org/pandas-docs/stable/text.html) en la documentación en línea de Pandas.

## Ejemplo: Base de datos de recetas

Estas operaciones de cadenas vectorizadas se vuelven más útiles en el proceso de limpieza de datos desordenados del mundo real. Aquí voy a mostrar un ejemplo de eso, usando una base de datos de recetas abierta compilada de varias fuentes en la Web. Nuestro objetivo será analizar los datos de la receta en listas de ingredientes, para que podamos encontrar rápidamente una receta basada en algunos ingredientes que tenemos a mano. 

Los scripts utilizados para compilar esto se pueden encontrar en https://github.com/fictivekin/openrecipes , y el enlace a la versión actual de la base de datos también se encuentra allí. 

A partir de la primavera de 2016, esta base de datos ocupa unos 30 MB y se puede descargar y descomprimir con estos comandos: 

In [ ]:
# !curl -O http://openrecipes.s3.amazonaws.com/recipeitems-latest.json.gz
# !gunzip recipeitems-latest.json.gz

La base de datos está en formato JSON, así que intentaremos ``pd.read_json`` para leerlo:

In [ ]:
try:
    recipes = pd.read_json('recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


¡UPS! obtenemos un ``ValueError`` mencionando que hay "datos de seguimiento". Buscando el texto de este error en Internet, parece que se debe a usar un archivo en el que cada línea es en sí misma un JSON válido, pero el archivo completo no lo es. Comprobemos si esta interpretación es cierta:

In [ ]:
with open('recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(line).shape

(2, 12)

Sí, aparentemente cada línea es un JSON válido, por lo que tendremos que unirlas. Una forma en que podemos hacer esto es construir una representación de cadena que contenga todas estas entradas JSON y luego cargar todo con ``pd.read_json``:

In [ ]:
# read the entire file into a Python array
with open('recipeitems-latest.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(data_json)

In [ ]:
recipes.shape

(173278, 17)

Vemos que hay casi 200.000 recetas y 17 columnas. Echemos un vistazo a una fila para ver lo que tenemos:

In [ ]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

Hay mucha información allí, pero gran parte de ella está en una forma muy desordenada, como es típico de los datos extraídos de la Web. En particular, la lista de ingredientes está en formato de cadena; vamos a tener que extraer cuidadosamente la información que nos interesa. Comencemos por echar un vistazo más de cerca a los ingredientes:

In [ ]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

¡La lista de ingredientes tiene un promedio de 250 caracteres, con un mínimo de 0 y un máximo de casi 10,000 caracteres!

Solo por curiosidad, veamos qué receta tiene la lista de ingredientes más larga:

In [ ]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

Eso ciertamente parece una receta complicada.

Podemos hacer otras exploraciones agregadas; por ejemplo, veamos cuántas de las recetas son de comida para el desayuno:

In [ ]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

O cuántas de las recetas mencionan la canela como ingrediente:

In [ ]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

Incluso podríamos ver si alguna receta escribe mal el ingrediente como "canela":

In [ ]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

Este es el tipo de exploración de datos esenciales que es posible con las herramientas de cadena de Pandas. Python realmente se destaca en la manipulación de datos como esta.

### Un sencillo recomendador de recetas

Avancemos un poco más y comencemos a trabajar en un sistema simple de recomendación de recetas: dada una lista de ingredientes, encuentre una receta que use todos esos ingredientes. Si bien es conceptualmente sencillo, la tarea se complica por la heterogeneidad de los datos: no existe una operación fácil, por ejemplo, para extraer una lista limpia de ingredientes de cada fila. Así que haremos un poco de trampa: comenzaremos con una lista de ingredientes comunes y simplemente buscaremos para ver si están en la lista de ingredientes de cada receta. Para simplificar, sigamos con las hierbas y especias por el momento: 

In [ ]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

Entonces podemos construir un booleano ``DataFrame`` que consta de valores Verdadero y Falso, indicando si este ingrediente aparece en la lista:

In [ ]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list))
spice_df.head()

,cumin,oregano,paprika,parsley,pepper,rosemary,sage,salt,tarragon,thyme
0,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Ahora, como ejemplo, digamos que nos gustaría encontrar una receta que use perejil, paprika y estragón. Podemos calcular esto muy rápidamente usando el método ``query()`` de ``DataFrames``.

In [ ]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

Encontramos solo 10 recetas con esta combinación; usemos el índice devuelto por esta selección para descubrir los nombres de las recetas que tienen esta combinación:

In [ ]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

Ahora que hemos reducido nuestra selección de recetas en un factor de casi 20 000, estamos en condiciones de tomar una decisión más informada sobre lo que nos gustaría cocinar para la cena.

### Yendo más allá con las recetas 

Con suerte, este ejemplo le ha dado un poco de sabor (¡ba-dum!) Para los tipos de operaciones de limpieza de datos que son habilitadas de manera eficiente por los métodos de cadena de Pandas. Por supuesto, construir un sistema de recomendación de recetas muy sólido requeriría mucho más trabajo! Extraer listas completas de ingredientes de cada receta sería una parte importante de la tarea; desafortunadamente, la amplia variedad de formatos utilizados hace que este sea un proceso relativamente lento. Esto apunta a la perogrullada de que, en la ciencia de datos, la limpieza y manipulación de datos del mundo real a menudo comprende la mayor parte del trabajo, y Pandas proporciona las herramientas que pueden ayudarlo a hacerlo de manera eficiente. 